In [1]:
# install the necessary extra libraries to run the script
#!apt-get update
#!apt-get install chromium chromium-driver
!pip install selenium
!pip install webdriver-manager
!pip install pandas

In [1]:
# import selenium, BeautifulSoup and other libraries to run the script
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from pandas import DataFrame
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from IPython.display import Image, display, clear_output
import re
import time
import pandas as pd
import os, shutil
import numpy as np
import undetected_chromedriver as uc
import urllib.request

# define constants to define the maximum waiting time and maximum amount of reviews to collect
MAX_WAIT = 10

In [2]:
# define a function to take a screenshot of the headless browser and display it
def take_screenshot(driver):
    driver.save_screenshot("scraping_folder/pageImage.png")
    display(Image(filename="scraping_folder/pageImage.png"))
    print()

In [3]:
# define a function to take a screenshot of the headless browser and display it, then clean the outcome after a few seconds
def take_screenshot_and_clear(driver):
    driver.save_screenshot("scraping_folder/pageImage.png")
    display(Image(filename="scraping_folder/pageImage.png"))
    
    time.sleep(3)
    clear_output(wait=True)

In [4]:
# define a function to move to the next page
def next_page(driver):   
    exists=1
    #check for the presence of privacy banner
    try:
        if driver.find_element(By.ID,'onetrust-accept-btn-handler') is not None:
            wait.until(EC.element_to_be_clickable((By.ID,'onetrust-accept-btn-handler'))).click()
    except Exception as e:
        print()
    #go to the next page  
    try:
        #get next button
        button=driver.find_element(By.CSS_SELECTOR, '.nav.next.ui_button.primary')
        if button.get_property('disabled')!=0:
            # wait until the next page button is clickable and then click it
            WebDriverWait(driver, MAX_WAIT).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.nav.next.ui_button.primary'))).click()
        else:
            exists=0
    except Exception as e:
        exists=0
        print()

    return exists

In [5]:
# define a function to
def expand_review(driver):
    # define an element to wait for an event to happen
    ex_wait = WebDriverWait(driver, MAX_WAIT)

    # load the complete review text in the HTML
    try:
        # wait until the element is clickable and when it is clickable, click it
        ex_wait.until(EC.element_to_be_clickable(
                    (By.XPATH,'//*[@id="taplc_location_reviews_list_resp_rr_resp_0"]/div/div[11]'))
                  ).click()

        # wait complete reviews to load
        time.sleep(5)
    # raised if there is no link for expansion (e.g.: set of short reviews)
    # IMPORTANT: nothing will be printed to avoid dirty outputs
    except Exception as e:
        #print(e)
        print()

In [6]:
def open_reviews(driver):
    #check for the presence of privacy banner
    try:
        if driver.find_element(By.ID,'onetrust-accept-btn-handler') is not None:
            wait.until(EC.element_to_be_clickable((By.ID,'onetrust-accept-btn-handler'))).click()
    except Exception as e:
        print()
    #click on "More"
    try:
        plus_button=driver.find_element(By.CSS_SELECTOR, '.taLnk.ulBlueLinks')
        ActionChains(driver).move_to_element(plus_button).click().perform()
        #wait complete reviews to load
        time.sleep(3)
    except Exception as e:
        print()

In [7]:
def get_review_data(resp, n_reviews, collected_data):
    take_screenshot(driver)
    
    # get the place's name
    name = resp.find('h1', class_ = 'HjBfq').text
    print(name)
            
    # collect the reviews
    r_list = resp.find_all('div', class_= 'reviewSelector')
    
    # for each review
    for idx, review in enumerate(r_list):
        #get id review
        id_review= review.get("data-reviewid")
        print(id_review)
        # get review date
        if review.find('div', class_='ui_column is-9') is not None:
            # the field may contain other textual contents, hence we only take the first 8 characters
            review_date = review.find('div', class_='ui_column is-9').find('span', class_='ratingDate').text
        else:
            # some reviews may have no date
            review_date = None

        print(review_date)
        #get user id
        user_id=None
        if review.find('div', class_='memberOverlayLink clickable') is not None:
            user_id=review.find('div', class_='memberOverlayLink clickable').get("id").split('-')[0]
        print(user_id)
        
        userLoc=None
        # get reviewer information: username, location
        if review.find('div', class_='info_text pointer_cursor') is not None:
            username=review.find('div', class_='info_text pointer_cursor').text
            if review.find('div', class_='info_text pointer_cursor').find("div", class_='userLoc') is not None:
                userLoc=review.find('div', class_='info_text pointer_cursor').find("div", class_='userLoc').text
                username=username.removesuffix(userLoc) 
        else:
            username='anonima'
        print(username)
        print(userLoc)
        #get number of reviews made by the user
        num_of_user_review=None
        if review.find('div', class_='reviewerBadge badge') is not None:
            if review.find('div', class_='reviewerBadge badge').find("span", class_='badgeText') is not None:
                num_of_user_review=review.find('div', class_='reviewerBadge badge').find("span", class_='badgeText').text
        elif review.find('div', class_='memberBadgingNoText is-shown-at-tablet') is not None:
            if review.find('div', class_='memberBadgingNoText is-shown-at-tablet').find("span", class_='badgetext') is not None:
                num_of_user_review=review.find('div', class_='memberBadgingNoText is-shown-at-tablet').find("span", class_='badgetext').text
        print(num_of_user_review)
        
        # get rating of review by looking at the aria-label of the svg with the dots       
        rating_review = review.find('div', class_='ui_column is-9').find("span", class_='ui_bubble_rating')
        rating_review=float(rating_review.get("class")[1].split('_')[1])
        rating_review=str(rating_review/10).replace(',', '.')
        print(rating_review)

        # get review title
        title = review.find('div', class_='quote').find('a', class_='title').text
        print(title)
        
        # get review complete text
        if review.find('div', class_='entry') is not None:                
            caption=review.find('div', class_='entry').text
            caption=caption.replace('Mostra meno','')
        print(caption)
        
        #get visit date
        if review.find('div', class_='prw_rup prw_reviews_stay_date_hsx') is not None:
            visit_date=review.find('div', class_='prw_rup prw_reviews_stay_date_hsx').text
        else:
            visit_date=None

        print(visit_date)
        
        quality_rating=None 
        atmosphere_rating=None 
        service_rating=None 
        food_rating=None 
        aspect_rated=0
        #get aspect ratings
        if review.find('div', class_='rating-list') is not None:
            if review.find('div', class_='rating-list').find('div', class_='recommend-description') is not None:
                elements=review.find('div', class_='rating-list').find_all('div', class_='recommend-description')
                bubble_ratings=review.find('div', class_='rating-list').find_all('div', class_='ui_bubble_rating')
                for elem in elements:
                    if elem.text=='Qualità/prezzo':
                        quality_rating=float(bubble_ratings[aspect_rated].get("class")[1].split('_')[1])
                        quality_rating=str(quality_rating/10).replace(',', '.')
                        aspect_rated+=1
                    elif elem.text=='Atmosfera':
                        atmosphere_rating=float(bubble_ratings[aspect_rated].get("class")[1].split('_')[1])
                        atmosphere_rating=str(atmosphere_rating/10).replace(',', '.')
                        aspect_rated+=1
                    elif elem.text=='Servizio':
                        service_rating=float(bubble_ratings[aspect_rated].get("class")[1].split('_')[1])
                        service_rating=str(service_rating/10).replace(',', '.')
                        aspect_rated+=1
                    else:
                        food_rating=float(bubble_ratings[aspect_rated].get("class")[1].split('_')[1])
                        food_rating=str(food_rating/10).replace(',', '.')
        
        print(quality_rating)  
        print(atmosphere_rating)
        print(service_rating)
        print(food_rating)
        
        # build review item
        item = {
            'id_review': id_review, # real id_review
            'name': name,
            'user_id': user_id,
            'userLoc': userLoc,
            'title': title,
            'caption': caption,
            'date': review_date,
            'rating': rating_review,
            'username': username,
            'visit_date': visit_date,
            'quality_rating': quality_rating,
            'atmosphere_rating': atmosphere_rating,
            'service_rating': service_rating,
            'food_rating': food_rating,
            'num_of_user_review': num_of_user_review
         }

         # count the number of reviews collected
        n_reviews = n_reviews + 1
        
        # add the item to the array of reviews collected
        collected_data.append(item)       

    time.sleep(5)
    clear_output(wait=True)

    # return the review count and the data collected
    return n_reviews, collected_data
    

In [25]:
# kill the chrome process to ensure a fresh start
!pkill chrome

# define the options to run Chrome
# IMPORTANT: these options are quite essential, especially when running it in a remote environment like Google Colab
options = webdriver.ChromeOptions()
# define the size of the window
options.add_argument("--window-size=1920,1080")
# disable notifications to avoid useless interactions 
options.add_argument("--disable-notifications")
# disable the developer options
options.add_argument('--disable-dev-shm-usage')
# set the browser to be headless (i.e., the browser window will be hidden)
options.add_argument('--headless')
# set the browser to run all scripts
options.add_argument('--no-sandbox')

service = Service(executable_path="./chromedriver.exe")
#service = Service()
driver = webdriver.Chrome(service=service, options=options)
#driver = webdriver.Chrome(service=Service(ChromeDriverManager(version='114.0.5735.90').install()),options=options)
#driver = webdriver.Chrome(ChromeDriverManager(version='114.0.5735.90').install(),options=options)
#driver = webdriver.Chrome(options=options)

# define an element to wait until an event occurs
wait = WebDriverWait(driver, MAX_WAIT)

#load urls file
file_url=pd.read_csv('Firenze/Firenze collected review.CSV', sep=';')
#file_url=pd.DataFrame(pd.read_csv('Firenze/Firenze collected review.CSV'), columns=['Urls', 'Number of Ratings', 'Details'])
num_of_urls=len(file_url)

for index in range(0, num_of_urls):
    #if the url has not yet been inspected
    if(file_url.loc[index,'Details']==0):
        #retrieve url
        url=file_url.loc[index,'Urls']
        # get the page with Selenium
        driver.get(url)
        
        # scroll and move to the filter section
        driver.execute_script('window.scrollBy(0,2500)')
        
        # expand the list of reviews (if needed)
        expand_review(driver)
    
        take_screenshot(driver)
        clear_output(wait=True)

        # wait for the page to be loaded
        WebDriverWait(driver, MAX_WAIT).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.memberOverlayLink.clickable')))
        
        #expand reviews
        open_reviews(driver)
        # send the page manipulated with Selenium to BeautifulSoup parser
        response = BeautifulSoup(driver.page_source, 'html.parser')
        
        n_reviews=0
        collected_data=[]
         
        name = response.find('h1', class_ = 'HjBfq').text
        pathname = name.replace('?','')
        pathname = pathname.replace('"','')
        pathname = pathname.replace('!...','')
        pathname = pathname.replace(':','')
        pathname = pathname.replace('/','')
        pathname = pathname.rstrip()
        #get the number of scraped reviews and the data collected
        n_reviews, collected_data = get_review_data(response, n_reviews, collected_data)
           
        while True:
            # move to the next page
            exists=next_page(driver) 
            if exists==0:
                print("if done")
                break
            # if there are still review pages
            # repeat the same process as before
            driver.execute_script('window.scrollBy(0,2500)')            
            
            expand_review(driver)

            WebDriverWait(driver, MAX_WAIT).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.memberOverlayLink.clickable')))              
            
            open_reviews(driver)
            response = BeautifulSoup(driver.page_source, 'html.parser') 
            
            n_reviews, collected_data = get_review_data(response, n_reviews, collected_data)
        #save reviews              
        df = DataFrame({'collected_data': collected_data})
        out_path = 'Firenze\\'+pathname+"\\"+pathname+" reviews.xlsx"
        writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
        df.to_excel(writer, sheet_name='sheet1')
        writer.save()
        #define the url as seen
        file_url.loc[index, 'Details'] = 1
        file_url.to_csv("Firenze/Firenze collected review.CSV", index=False)
        time.sleep(2)
        clear_output(wait=True)

# close the driver
driver.close()
driver.quit()



if done
